In [7]:
import numpy as np  
import pandas as pd  
from bs4 import BeautifulSoup
import pickle
import os, re

In [8]:
# from nltk.corpus import stopwords
train = pd.read_csv("../Dataset/word2vec-nlp/labeledTrainData.tsv", header=0, delimiter='\t')
test = pd.read_csv("../Dataset/word2vec-nlp/testData.tsv", header=0, delimiter='\t')
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = test["sentiment"]

In [9]:
def html_to_text(review):
    """Return extracted text string from provided HTML string."""
    review_text = BeautifulSoup(review, "lxml").get_text()
    if len(review_text) == 0:
        review_text = review
    review_text = re.sub(r"\<.*\>", "", review_text)
    try:
        review_text = review_text.encode('ascii', 'ignore').decode('ascii')  # ignore \xc3 etc.
    except UnicodeDecodeError:
        review_text = review_text.decode("ascii", "ignore")
    return review_text


def letters_only(text):
    """Return input string with only letters (no punctuation, no numbers)."""
    # It is probably worth experimenting with milder prepreocessing (eg just removing punctuation)
    return re.sub("[^a-zA-Z]", " ", text)


def rnn_tokenizer_review_preprocess(review):
    """Preprocessing used before fitting/transforming RNN tokenizer - Html->text, remove punctuation/#s, lowercase."""
    return letters_only(html_to_text(review)).lower()


def get_train_val_data(reviews_to_features_fn=None, df=train):
    """Extracts features (using reviews_to_features_fn), splits into train/test data, and returns
    x_train, y_train, x_test, y_test.  If no feature extraction function is provided, x_train/x_test will
    simply consist of a Series of all the reviews.
    """
    #     df = pd.read_csv('labeledTrainData.tsv', header=0, quotechar='"', sep='\t')
    SEED = 1000
    # Shuffle data frame rows
    np.random.seed(SEED)
    df = df.iloc[np.random.permutation(len(df))]

    if reviews_to_features_fn:
        feature_rows = df["review"].map(reviews_to_features_fn)
        if type(feature_rows[0]) == np.ndarray:
            num_instances = len(feature_rows)
            num_features = len(feature_rows[0])
            x = np.concatenate(feature_rows.values).reshape((num_instances, num_features))
        else:
            x = feature_rows
    else:
        x = df["review"]

    y = df["sentiment"]

    # Split 80/20
    test_start_index = int(df.shape[0] * .8)
    x_train = x[0:test_start_index]
    y_train = y[0:test_start_index]
    x_val = x[test_start_index:]
    y_val = y[test_start_index:]

    return x_train, y_train, x_val, y_val

In [10]:
import time

x_train, y_train, x_val, y_val = get_train_val_data(rnn_tokenizer_review_preprocess)

x_test = test["review"].map(rnn_tokenizer_review_preprocess)
y_test = test["sentiment"]

# ----- lex -----

pos_list = []
neg_list = []
with open("../Dataset/SentiWordNet/SentiWordNet_3.0.0.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        if line[0] != '#':
            # print(line)
            strarr = line.split('\t')
            # print(strarr)
            # print(strarr[2].split("\n")[0])
            if (strarr[2] != '') & (strarr[2] != ' ') & (strarr[2] != '#'):
                if float(strarr[2].split("\n")[0]) > float(strarr[3].split("\n")[0]):
                    pos_list += [strarr[4].split('#')[0]]
                if float(strarr[2].split("\n")[0]) < float(strarr[3].split("\n")[0]):
                    neg_list += [strarr[4].split('#')[0]]

# print(pos_list)
# print(neg_list)

In [11]:
pos = 0
neg = 0

pre = -1
# print(len(x_test))

from joblib import Parallel, delayed


def comp(str, pos_list, neg_list, pos, neg):
    if str in pos_list:
        pos += 1
    if str in neg_list:
        neg += 1
    return


start_time = time.time()


# for index in range(1000):
#     str_arr = x_test[index].split(' ')
#     Parallel(n_jobs=4)(delayed(comp)(str,pos_list,neg_list,pos,neg) for str in str_arr)
#     if pos >= neg:
#         pre = 1
#     else:
#         pre = 0
#     pos=0
#     neg=0
#     if pre == y_test[index]:
#         predict_correct+=1
#     else:
#         predict_error+=1
predict_correct = 0
predict_error = 0

def par_comp(mark, index):
    pos = 0
    neg = 0
    str_arr = x_test[index].split(' ')
    for i in str_arr:
        if i in pos_list:
            pos += 1
        if i in neg_list:
            neg += 1
    if pos >= neg:
        pre = 1
    else:
        pre = 0

    if pre == y_test[index]:
        mark += 1
    else:
        mark -= 1

    return mark

mark = 0
mark = Parallel(n_jobs=2)(
    delayed
    (par_comp)
    (mark, index)
    for index
    in range(100))


#
# for index in range(1000):
#     str_arr = x_test[index].split(' ')
#     for i in str_arr:
#         if i in pos_list:
#             pos+=1
#         if i in neg_list:
#             neg+=1
#     if pos >= neg:
#         pre = 1
#     else:
#         pre = 0
#     pos=0
#     neg=0
#     if pre == y_test[index]:
#         predict_correct+=1
#     else:
#         predict_error+=1
print("time cost:", time.time() - start_time)
for i in mark:
    if i == 1:
        predict_correct+=1
    else:
        predict_error+=1
print("Prediction:")
print(predict_correct)
print(predict_error)






# print(pos)
# print(neg)
# print(y_test[0])


#
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
#
#
# np.random.seed(1000)
# num_most_freq_words_to_include = 5000
# MAX_REVIEW_LENGTH_FOR_KERAS_RNN = 500
# embedding_vector_length = 32
# # train_review_list = [s.encode('ascii') for s in x_train.tolist()]
# # val_review_list = [s.encode('ascii') for s in x_val.tolist()]
# # all_review_list = train_review_list + val_review_list
# train_review_list = x_train.tolist()
# val_review_list = x_val.tolist()
# test_review_list = x_test.tolist()
# all_review_list = x_train.tolist() + x_val.tolist()
# tokenizer = Tokenizer(num_words=num_most_freq_words_to_include)
# tokenizer.fit_on_texts(all_review_list)
# train_reviews_tokenized = tokenizer.texts_to_sequences(train_review_list)
# x_train = pad_sequences(train_reviews_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
# val_review_tokenized = tokenizer.texts_to_sequences(val_review_list)
# x_val = pad_sequences(val_review_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
# test_review_tokenized = tokenizer.texts_to_sequences(test_review_list)
# x_test = pad_sequences(test_review_tokenized, maxlen=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)

#
# from keras.layers import Input, Embedding, Dropout, Conv1D, MaxPool1D, GRU, LSTM, Dense
# from keras.models import Model
#
#
# def rnn_model(use_cnn=True, use_lstm=False):
#     input_sequences = Input(shape=(MAX_REVIEW_LENGTH_FOR_KERAS_RNN,))
#     initial_dropout = 0.2
#     embedding_layer = Embedding(input_dim=num_most_freq_words_to_include,
#                                 output_dim=embedding_vector_length,
#                                 input_length=MAX_REVIEW_LENGTH_FOR_KERAS_RNN)
#     X = embedding_layer(input_sequences)
#     X = Dropout(0.2)(X)
#     if use_cnn:
#         X = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(X)
#         X = MaxPool1D(pool_size=2)(X)
#
#     # Add GRU layers
#     dropout_W = 0.0
#     dropout_U = 0.0
#
#     if use_lstm:
#         X = LSTM(100, dropout=dropout_W, recurrent_dropout=dropout_U)(X)
#     else:
#         X = GRU(100, dropout=dropout_W, recurrent_dropout=dropout_U)(X)
#     X = Dropout(0.2)(X)
#     outputs = Dense(1, activation='sigmoid')(X)
#     model = Model(inputs=input_sequences, outputs=outputs)
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#
#     return model
#
#
#
# gru_model = rnn_model(use_lstm=False)
# gru_model.summary()


time cost: 52.7130651473999
Prediction:
51
49
